In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import time

In [2]:
#训练集
uid_train = pd.read_csv('uid_train.txt',sep='\t',header=None,names=('uid','label'))

voice_train = pd.read_csv('voice_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})

sms_train = pd.read_csv('sms_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})

wa_train = pd.read_csv('wa_train.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
wa_train.isnull().sum()

uid             0
wa_name       801
visit_cnt     801
visit_dura    801
up_flow       801
down_flow     801
wa_type       801
date          801
dtype: int64

In [4]:
#测试集
voice_test = pd.read_csv('voice_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})

sms_test = pd.read_csv('sms_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})

wa_test = pd.read_csv('wa_test_b.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

In [5]:
wa_test.isnull().sum()

uid             0
wa_name       497
visit_cnt     497
visit_dura    497
up_flow       497
down_flow     497
wa_type       497
date          497
dtype: int64

In [6]:
uid_test = pd.DataFrame({'uid':pd.unique(wa_test['uid'])})
uid_test.to_csv('uid_test_b.txt',index=None)

In [7]:
voice = pd.concat([voice_train,voice_test],axis=0)
sms = pd.concat([sms_train,sms_test],axis=0)
wa = pd.concat([wa_train,wa_test],axis=0)

In [26]:
#voice.isnull().sum()
voice.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1760353 entries, 0 to 609574
Data columns (total 8 columns):
uid           object
opp_num       object
opp_head      object
opp_len       int64
start_time    object
end_time      object
call_type     int64
in_out        int64
dtypes: int64(3), object(5)
memory usage: 160.9+ MB


In [9]:
sms.isnull().sum()

uid           0
opp_num       0
opp_head      0
opp_len       0
start_time    0
in_out        0
dtype: int64

In [10]:
wa.isnull().sum()

uid              0
wa_name       1298
visit_cnt     1298
visit_dura    1298
up_flow       1298
down_flow     1298
wa_type       1298
date          1298
dtype: int64

In [11]:
wa = wa.fillna(-999)
wa.isnull().sum()

uid           0
wa_name       0
visit_cnt     0
visit_dura    0
up_flow       0
down_flow     0
wa_type       0
date          0
dtype: int64

In [12]:
voice.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1760353 entries, 0 to 609574
Data columns (total 8 columns):
uid           object
opp_num       object
opp_head      object
opp_len       int64
start_time    object
end_time      object
call_type     int64
in_out        int64
dtypes: int64(3), object(5)
memory usage: 120.9+ MB


In [49]:
#通话记录
voice_opp_num = voice.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x:len(set(x)), 'count':lambda x:x.count()}).add_prefix('voice_opp_num').reset_index().fillna(0)

voice_opp_head = voice.groupby(['uid'])['opp_head'].agg({'unique_count':lambda x:len(set(x))}).add_prefix('voice_opp_head').reset_index().fillna(0)
#head_3 = voice.loc[voice['opp_head'] >= 100].groupby(['uid'])['opp_head'].agg({'count_3':lambda x:x.count()}).reset_index().fillna(0)
#print head_3
#head_1 = voice.loc[voice['opp_head'] < 100].groupby(['uid'])['opp_head'].agg({'count_1':lambda x:x.count()}).reset_index().fillna(0)
#head_1.info()

voice_opp_len = voice.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('voice_opp_len').reset_index().fillna(0)

voice_call_type = voice.groupby(['uid','call_type'])['uid'].count().unstack().add_prefix('voice_call_type').reset_index().fillna(0)

voice_in_out = voice.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('voice_in_out').reset_index().fillna(0)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


call_type                                                  1  \
uid                                                            
u0001      D1DE6D77FF45AC7E143522C831111AA8E16C88EC4C9A2D...   
u0002      B49D3D47438C2C5E7B48FF7245D1EB73435B023764E541...   
u0003      640A47F46D750A464108C6FB72AFDE1A7D64E64514307C...   
u0004      86F2E03E7E60D72476475F205673A01B98D8BB7C6CDF69...   
u0005      5055F72FBFA9060FC6414EA07E749A13D42CA10F055BC4...   
u0006      CEE5FEB134D2B62AA5A77EECCAE29F9005FC4673D1FC2C...   
u0007      F601CF228617364D6D4E6CD23A63ED6D7A0639DB5609BC...   
u0008      CBDC885E4E3B4BA45CE493D9F498A9D7C93B5086CEA244...   
u0009      CE20BD2CAECE2820DE66419D7225CAC4979E724A0964B8...   
u0010      E1901B64A9A9CB1D0A9CE1E457B736EF1E6B35BFC6CAE8...   
u0011      45D385915C242D775818C96AFB2BCBCA27DC23A04E6E59...   
u0012      5E1D6C23859206AA7EB54047CE05661620AA93774F8D99...   
u0013      C7EA0EDD524086C369CFB9C0412EB774F506440BC38DCE...   
u0014      F19A008ADA30EB90B7D4B0AC0E5FB

In [16]:
#短信记录
sms_opp_num = sms.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(set(x)),'count':'count'}).add_prefix('sms_opp_num_').reset_index().fillna(0)

sms_opp_head = sms.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(set(x))}).add_prefix('sms_opp_head_').reset_index().fillna(0)

sms_opp_len=sms.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('sms_opp_len_').reset_index().fillna(0)

sms_in_out = sms.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('sms_in_out_').reset_index().fillna(0)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


In [17]:
#网站/app记录
wa_name = wa.groupby(['uid'])['wa_name'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('wa_name_').reset_index().fillna(0)

visit_cnt = wa.groupby(['uid'])['visit_cnt'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_cnt_').reset_index().fillna(0)

#根据特征重要性图删掉了min这个特征值
visit_dura = wa.groupby(['uid'])['visit_dura'].agg(['std','max','median','mean','sum']).add_prefix('wa_visit_dura_').reset_index().fillna(0)

up_flow = wa.groupby(['uid'])['up_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_up_flow_').reset_index().fillna(0)

down_flow = wa.groupby(['uid'])['down_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_down_flow_').reset_index().fillna(0)

#增加wa类型特征
wa_type = wa.groupby(['uid','wa_type'])['uid'].count().unstack().add_prefix('wa_type').reset_index().fillna(0)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [18]:
feature = [voice_opp_num,voice_opp_head,voice_opp_len,voice_call_type,voice_in_out,sms_opp_num,sms_opp_head,sms_opp_len,sms_in_out,wa_name,visit_cnt,visit_dura,up_flow,
           down_flow,wa_type]

In [19]:
train_feature = uid_train
for feat in feature:
    train_feature=pd.merge(train_feature,feat,how='left',on='uid')

In [20]:
test_feature = uid_test
for feat in feature:
    test_feature=pd.merge(test_feature,feat,how='left',on='uid')

In [21]:
train_feature.to_csv('train_featurev2.csv',index=None)
test_feature.to_csv('test_featurev2.csv',index=None)